In [1]:
import os
import json
import pandas as pd

In [2]:
# Set the base directory for the simulation results
data_dir = os.path.join(os.getcwd(), 'simulation_results')

# Define the list of scenarios and CSV types
scenarios = ['clear-night', 'clear-noon', 'clear-sunset', 'haze-noon', 'haze-sunset', 'rain-noon']
csvs = ['ctl', 'cvip', 'traj']

In [3]:
# Function to read the route_highway.txt file
def read_route_highway_txt(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        # Extract necessary fields from JSON-like data
        return {
            'duration_game': data['_checkpoint']['records'][0]['meta']['duration_game'],
            'duration_system': data['_checkpoint']['records'][0]['meta']['duration_system'],
            'route_length': data['_checkpoint']['records'][0]['meta']['route_length'],
            'score_composed': data['_checkpoint']['records'][0]['scores']['score_composed'],
            'score_penalty': data['_checkpoint']['records'][0]['scores']['score_penalty'],
            'score_route': data['_checkpoint']['records'][0]['scores']['score_route'],
            'status': data['_checkpoint']['records'][0]['status']
        }

# Function to read CSV files
def read_csv_file(file_path):
    return pd.read_csv(file_path)

In [4]:
df_list = []

In [5]:
# Iterate over each scenario
for scenario in scenarios:
    folder_path = os.path.join(data_dir, f'route_highway_epoch24_{scenario}_fi_ghost_cutin')
    files_in_folder = os.listdir(folder_path)
    print(f"Files in folder {folder_path}: {files_in_folder}")

    # Read the route_highway.txt file
    route_highway_txt_path = os.path.join(folder_path, 'route_highway.txt')

    if os.path.exists(route_highway_txt_path):
        route_highway_data = read_route_highway_txt(route_highway_txt_path)
        route_highway_df = pd.DataFrame([route_highway_data])
    else:
        print(f"route_highway.txt not found in {folder_path}")
        continue

    # Initialize an empty dictionary to hold CSV data for each scenario
    csv_data_dict = {}
    # Read the CSV files (ctl, cvip, traj)
    for csv_type in csvs:
        matching_files = [f for f in files_in_folder if f.endswith(f'single_{csv_type}.csv')]
        if matching_files:
            csv_file_path = os.path.join(folder_path, matching_files[0])
            csv_df = read_csv_file(csv_file_path)
            csv_data_dict[csv_type] = csv_df
        else:
            print(f"No file ending with 'single_{csv_type}.csv' found in {folder_path}")

    # Merge CSV dataframes on 'ts','agent_id' column
    if 'ctl' in csv_data_dict:
        merged_csv_df = csv_data_dict['ctl']
        for csv_type in ['cvip', 'traj']:
            if csv_type in csv_data_dict:
                merged_csv_df = pd.merge(merged_csv_df, csv_data_dict[csv_type], on=['ts','agent_id'], how='left')

        # Concatenate the route_highway data to match the length of merged CSV dataframe
        full_df = pd.concat([merged_csv_df, pd.concat([route_highway_df]*len(merged_csv_df), ignore_index=True)], axis=1)
        full_df['scenario'] = scenario
        df_list.append(full_df)

Files in folder c:\Users\cindy\Desktop\Mini-Project-1-Safety-Analysis-of-Autonomous-Vehicles\simulation_results\route_highway_epoch24_clear-night_fi_ghost_cutin: ['routes_fi_route_highway-0915_005809-data_fi_ghost_cutin-single_ctl.csv', 'routes_fi_route_highway-0915_005809-data_fi_ghost_cutin-single_cvip.csv', 'routes_fi_route_highway-0915_005809-data_fi_ghost_cutin-single_traj.csv', 'route_highway.txt', 'run.done']
Files in folder c:\Users\cindy\Desktop\Mini-Project-1-Safety-Analysis-of-Autonomous-Vehicles\simulation_results\route_highway_epoch24_clear-noon_fi_ghost_cutin: ['routes_fi_route_highway-0915_005230-data_fi_ghost_cutin-single_ctl.csv', 'routes_fi_route_highway-0915_005230-data_fi_ghost_cutin-single_cvip.csv', 'routes_fi_route_highway-0915_005230-data_fi_ghost_cutin-single_traj.csv', 'route_highway.txt', 'run.done']
Files in folder c:\Users\cindy\Desktop\Mini-Project-1-Safety-Analysis-of-Autonomous-Vehicles\simulation_results\route_highway_epoch24_clear-sunset_fi_ghost_cutin

In [6]:
# Combine all scenario data into one dataframe
if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
else:
    print("No valid dataframes were created.")

In [7]:
final_df['scenario'].value_counts()

scenario
clear-night     860
clear-sunset    757
haze-sunset     757
haze-noon       756
clear-noon      753
rain-noon       402
Name: count, dtype: int64

In [8]:
final_df.head()

,ts,agent_id,throttle,steer,brake,cvip,cvip_x,cvip_y,cvip_z,x,...,z,v,duration_game,duration_system,route_length,score_composed,score_penalty,score_route,status,scenario
0,22555,0,0.9,-0.012952,0.0,500.491189,198.767441,-95.832657,-499.819366,192.362411,...,0.539326,0.0,21.5,73.414752,131.560126,100.0,1.0,100.0,Completed,clear-night
1,22556,0,0.9,-0.003071,0.0,5.595580,195.567444,-90.832657,0.100000,192.362411,...,0.491906,0.0,21.5,73.414752,131.560126,100.0,1.0,100.0,Completed,clear-night
2,22557,0,0.9,-0.010238,0.0,5.592365,195.567444,-90.832657,0.095407,192.362411,...,0.438374,0.0,21.5,73.414752,131.560126,100.0,1.0,100.0,Completed,clear-night
3,22558,0,0.9,-0.008100,0.0,5.589578,195.567444,-90.832657,0.084691,192.362411,...,0.378732,0.0,21.5,73.414752,131.560126,100.0,1.0,100.0,Completed,clear-night
4,22559,0,0.9,-0.000104,0.0,5.587154,195.567444,-90.832657,0.069311,192.362411,...,0.312981,0.0,21.5,73.414752,131.560126,100.0,1.0,100.0,Completed,clear-night


In [9]:
final_df.columns

Index(['ts', 'agent_id', 'throttle', 'steer', 'brake', 'cvip', 'cvip_x',
       'cvip_y', 'cvip_z', 'x', 'y', 'z', 'v', 'duration_game',
       'duration_system', 'route_length', 'score_composed', 'score_penalty',
       'score_route', 'status', 'scenario'],
      dtype='object')

In [10]:
final_df.to_csv('combined_df.csv', index=False)